# Data Cleaning Fundamentals

## Learning Objectives
- Understand common data quality issues
- Learn techniques for handling missing data
- Master data type conversions
- Remove duplicates and outliers

## Prerequisites
- Basic Python knowledge
- Pandas fundamentals

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## 1. Creating Sample Messy Data

In [ ]:
# Create a messy dataset for demonstration
np.random.seed(42)

data = {
    'name': ['Alice', 'Bob', 'Charlie', 'Alice', 'Eve', None, 'Frank', 'Grace'],
    'age': [25, 30, None, 25, 35, 28, 40, 22],
    'salary': [50000, 60000, 70000, 50000, None, 55000, 80000, 45000],
    'department': ['IT', 'HR', 'IT', 'IT', 'Finance', 'HR', 'IT', 'Finance'],
    'join_date': ['2020-01-15', '2019-03-20', '2021-07-10', '2020-01-15', 
                  '2018-11-05', '2020-09-12', '2017-04-18', '2022-02-28']
}

df = pd.DataFrame(data)
print("Original Dataset:")
print(df)
print(f"\nDataset shape: {df.shape}")

## 2. Data Quality Assessment

In [ ]:
# Check data info
print("Dataset Info:")
print(df.info())

print("\nMissing Values:")
print(df.isnull().sum())

print("\nDuplicate Rows:")
print(f"Number of duplicates: {df.duplicated().sum()}")

print("\nData Types:")
print(df.dtypes)

## 3. Handling Missing Data

In [ ]:
# Strategy 1: Drop rows with missing values
df_dropped = df.dropna()
print("After dropping missing values:")
print(df_dropped)
print(f"Shape: {df_dropped.shape}")

In [ ]:
# Strategy 2: Fill missing values
df_filled = df.copy()

# Fill missing names with 'Unknown'
df_filled['name'].fillna('Unknown', inplace=True)

# Fill missing age with median
df_filled['age'].fillna(df_filled['age'].median(), inplace=True)

# Fill missing salary with mean
df_filled['salary'].fillna(df_filled['salary'].mean(), inplace=True)

print("After filling missing values:")
print(df_filled)
print(f"\nMissing values: {df_filled.isnull().sum().sum()}")

## 4. Removing Duplicates

In [ ]:
# Remove duplicate rows
df_no_duplicates = df_filled.drop_duplicates()
print("After removing duplicates:")
print(df_no_duplicates)
print(f"Shape: {df_no_duplicates.shape}")

## 5. Data Type Conversions

In [ ]:
# Convert join_date to datetime
df_clean = df_no_duplicates.copy()
df_clean['join_date'] = pd.to_datetime(df_clean['join_date'])

# Convert age to integer
df_clean['age'] = df_clean['age'].astype(int)

# Convert salary to integer
df_clean['salary'] = df_clean['salary'].astype(int)

print("Final cleaned dataset:")
print(df_clean)
print(f"\nData types:")
print(df_clean.dtypes)

## 6. Outlier Detection and Handling

In [ ]:
# Detect outliers using IQR method
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Check for outliers in salary
salary_outliers, lower, upper = detect_outliers_iqr(df_clean, 'salary')
print(f"Salary outliers (< {lower:.0f} or > {upper:.0f}):")
print(salary_outliers)

# Visualize outliers
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.boxplot(df_clean['salary'])
plt.title('Salary Distribution (Box Plot)')
plt.ylabel('Salary')

plt.subplot(1, 2, 2)
plt.hist(df_clean['salary'], bins=10, edgecolor='black')
plt.title('Salary Distribution (Histogram)')
plt.xlabel('Salary')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## 7. Data Validation

In [ ]:
# Validate data ranges and constraints
def validate_data(df):
    issues = []
    
    # Check age range
    if (df['age'] < 18).any() or (df['age'] > 100).any():
        issues.append("Age values outside reasonable range (18-100)")
    
    # Check salary range
    if (df['salary'] < 0).any():
        issues.append("Negative salary values found")
    
    # Check for future join dates
    if (df['join_date'] > pd.Timestamp.now()).any():
        issues.append("Future join dates found")
    
    return issues

validation_issues = validate_data(df_clean)
if validation_issues:
    print("Data validation issues:")
    for issue in validation_issues:
        print(f"- {issue}")
else:
    print("✅ Data validation passed!")

## 8. Summary Statistics

In [ ]:
# Generate summary statistics
print("Summary Statistics:")
print(df_clean.describe())

print("\nCategorical Data Summary:")
print(df_clean['department'].value_counts())

## 🎯 Key Takeaways

1. **Always assess data quality first** - Check for missing values, duplicates, and data types
2. **Choose appropriate strategies** - Different missing data strategies for different scenarios
3. **Handle outliers carefully** - Understand if they're errors or valid extreme values
4. **Validate your cleaning** - Ensure the cleaned data makes business sense
5. **Document your process** - Keep track of all transformations for reproducibility

## 📝 Exercises

1. Create a dataset with different types of missing data patterns
2. Implement forward-fill and backward-fill strategies
3. Compare different outlier detection methods (Z-score, IQR, Isolation Forest)
4. Build a data cleaning pipeline function